In [4]:
import pandas as pd
from sklearn import preprocessing
from collections import defaultdict
import numpy as np
import time
import qgrid

In [5]:
def loadCSV(pathSamples, pathMatrix):
    df_loaded = pd.read_table(pathMatrix, sep=',')
    data = pd.read_table(pathSamples, sep=',')
    clin_trial_values = df_loaded.values
    columns = df_loaded.columns
    
    Y = data['Eligible']
    Y = Y.astype(int)
    X = clin_trial_values[:, :]
    return X,Y, columns

In [8]:
X, Y, columns = loadCSV("../../Dataset/10k_1Col_NoCarEsp_LSA.csv", "../../Tables/docsTopicsLSA1500.csv") #Cargar SCV

In [9]:
from sklearn.model_selection import train_test_split 
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=.2) # Modificar metodología de validación

In [56]:
import xgboost
from sklearn.metrics import accuracy_score

def xgBoost(X,Y,learning_rate=0.1,  colsample_bytree = 0.3, max_depth = 5, n_estimators = 20, reg_alpha = 10):
    
    Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, Y, test_size=.2)
    time_finish = time.time()
    # cargamos las 4 combinaciones de las compuertas XOR
    training_data = Xtrain

    # y estos son los resultados que se obtienen, en el mismo orden
    target_data = Ytrain
    
    model = xgboost.XGBClassifier(objective ='binary:logistic', colsample_bytree = colsample_bytree, learning_rate = learning_rate,
                    max_depth = max_depth, n_estimators = n_estimators, booster = 'gbtree', reg_alpha = reg_alpha)


    model.fit(training_data, target_data, eval_metric='logloss')
    
    preds = model.predict(Xtest)
    pred_train = model.predict(training_data)
    
    accuracyTrain = accuracy_score(target_data.values, pred_train)
    
    accuracyTest = accuracy_score(Ytest.values, preds)

    return model, str(time.time() - time_finish), accuracyTrain, accuracyTest

In [57]:
pd.options.mode.chained_assignment = None
import time
import qgrid

randn = np.random.randn
df_types = pd.DataFrame({
    'learning_rate' : pd.Series([0.2,0.2,0.15,0.3]), 
    'colsample_bytree' : pd.Series([0.4,0.8,0.6,0.3]), 
    'max_depth' : pd.Series([12,10,3,4]),
    'reg_alpha' : pd.Series([50,100,50,0]),
    'n_estimators' : pd.Series([40,40,10,20])})
df_types["Accuracy train"] = ""
df_types["Accuracy test"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['learning_rate', 'colsample_bytree', 'max_depth', 'n_estimators', 'reg_alpha'], inplace=True)

for learning_rate, colsample_bytree, max_depth, n_estimators, reg_alpha in df_types.index:
    modelo, time_finish, accuracyTrain, accuracyTest = xgBoost(X,Y,learning_rate=learning_rate,colsample_bytree = colsample_bytree,
                                                              max_depth = max_depth, n_estimators = n_estimators, reg_alpha = reg_alpha)   
    
    
    df_types["Accuracy test"][learning_rate, colsample_bytree, max_depth, n_estimators, reg_alpha] = (accuracyTest * 100.0)
    df_types["Tiempo de ejecución"][learning_rate, colsample_bytree, max_depth,n_estimators, reg_alpha] = time_finish
    df_types["Accuracy train"][learning_rate, colsample_bytree, max_depth, n_estimators, reg_alpha] = (accuracyTrain * 100.0)
    
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget.get_changed_df()

Accuracy train  \
learning_rate colsample_bytree max_depth n_estimators reg_alpha                  
0.20          0.4              12        40           50                 89.25   
              0.8              10        40           100               80.825   
0.15          0.6              3         10           50                68.425   
0.30          0.3              4         20           0                  82.85   

                                                                Accuracy test  \
learning_rate colsample_bytree max_depth n_estimators reg_alpha                 
0.20          0.4              12        40           50                75.25   
              0.8              10        40           100               71.35   
0.15          0.6              3         10           50                67.45   
0.30          0.3              4         20           0                  71.5   

                                                                Tiempo de ejecución  
learning_rate colsample_bytree max_depth n_estimators reg_alpha                      
0.20          0.4              12        40           50          7.038994550704956  
              0.8              10        40           100         8.253281354904175  
0.15          0.6              3         10           50          1.327953815460205  
0.30          0.3              4         20           0          1.5586626529693604